Library imports.

In [38]:
import json
from PIL import Image
import numpy as np

def load_image_as_bytes(infilename) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    np_array_to_list = data.tolist()
    b = bytes(json.dumps(np_array_to_list), 'utf-8')
    return b

from kafka import KafkaProducer
from kafka import KafkaConsumer

Create the topic.

In [41]:
from kafka.admin import KafkaAdminClient, NewTopic
admin_client = KafkaAdminClient(bootstrap_servers=['10.0.0.14'], client_id='test')
topic_list = []
topic_list.append(NewTopic(name="test", num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

CreateTopicsResponse_v2(throttle_time_ms=0, topic_errors=[(topic='test', error_code=0, error_message=None)])

Load the image that we want to send.

In [42]:
b = load_image_as_bytes('download.jpeg')

Send it to the cluster.

In [44]:
# Replace the `ip_address` entries with the IP address of your worker nodes
# NOTE: you don't need the full list of worker nodes, just one or two.
producer = KafkaProducer(bootstrap_servers=['10.0.0.14'])
producer.send('test', b)

Read it from the cluster.

In [45]:
# Replace the `ip_address` entries with the IP address of your worker nodes
# Again, you only need one or two, not the full list.
# Note: auto_offset_reset='earliest' resets the starting offset to the beginning
#       of the topic
consumer = KafkaConsumer(bootstrap_servers=['10.0.0.14', '10.0.0.13'],auto_offset_reset='earliest')
consumer.subscribe(['test'])
for msg in consumer:
    print (msg)


ConsumerRecord(topic='test', partition=0, offset=0, timestamp=1557605289526, timestamp_type=0, key=None, value=b'[[[230, 134, 118], [230, 134, 118], [229, 133, 117], [229, 133, 117], [228, 133, 115], [228, 133, 115], [228, 133, 115], [228, 133, 113], [229, 134, 114], [227, 132, 112], [224, 129, 107], [221, 126, 104], [220, 125, 103], [221, 126, 104], [223, 128, 106], [224, 130, 105], [225, 141, 115], [228, 142, 117], [232, 146, 119], [235, 148, 121], [239, 147, 122], [240, 146, 121], [240, 144, 122], [240, 141, 122], [220, 119, 107], [210, 108, 103], [191, 91, 93], [175, 74, 82], [165, 64, 80], [161, 62, 83], [164, 64, 90], [167, 66, 96], [178, 74, 85], [178, 74, 85], [178, 74, 85], [179, 75, 86], [179, 75, 84], [179, 75, 84], [180, 76, 85], [180, 76, 83], [180, 76, 83], [179, 75, 82], [178, 75, 79], [178, 75, 79], [180, 77, 81], [184, 81, 85], [188, 85, 89], [190, 87, 90], [196, 91, 88], [196, 91, 88], [197, 92, 89], [199, 94, 91], [201, 96, 93], [202, 97, 94], [203, 98, 95], [204, 99

KeyboardInterrupt: 